In [1]:
import torch
import torch.nn as nn
import time

In [3]:
class model_target(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1 = nn.Conv2d(1,64,(3,1),padding=(1,0))
        self.bn1 = nn.BatchNorm2d(64)
        self.act1 = nn.Sigmoid()
        self.c2 = nn.Conv2d(64,256,(3,1),padding=(1,0))
        self.c3 = nn.Conv2d(256,1,(3,1),padding=(1,0))
        self.fc = torch.nn.Linear(257, 257)

    def forward(self, x):
        #print(x.shape)
        x = self.c1(x)
        x = self.bn1(x)
        x = self.act1(x)
        #print(x.shape)
        x = self.c2(x)
       # print(x.shape)
        x = self.c3(x)
        x = x.permute((0,1,3,2))
       # print(x.shape)
        x = self.fc(x)
        return x

def test(m,shape=(2,1,257,8), N = 100) : 
    T = 0.0
    for i in range(N) :
        x = torch.rand(shape)
        tic = time.time()
        y = m(x)
        toc = time.time()
        elapsed = toc-tic
        T += elapsed
    return T/N

m_fp32 = model_target()
m_fp32.eval()
m_int8 = torch.ao.quantization.quantize_dynamic(
    m_fp32,  # the original model
    {torch.nn.Linear, torch.nn.Conv2d},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights


print("fp32")
print(test(m_fp32))
print("int8")
print(test(m_int8))

m_fp32.qconfig = torch.ao.quantization.get_default_qconfig('x86')
m_fp32 = model_fp32_prepared = torch.ao.quantization.prepare(m_fp32)
m_int8 = torch.ao.quantization.convert(m_fp32)

print("fp32")
print(test(m_fp32))
print("int8")
print(test(m_int8))

fp32
0.0027522635459899902
int8
0.0029647088050842284
fp32
0.006040494441986084
int8


NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered at ../aten/src/ATen/core/MetaFallbackKernel.cpp:23